In [ ]:
import random
import math

# Step 1: Define the new optimization problem function
def fitness_function(x):
    return math.sin(x) * x**2

# Step 2: Initialize parameters
population_size = 10
mutation_rate = 0.1
crossover_rate = 0.7
generations = 10  # Reduced number of generations to 10
gene_length = 10  # Length of the binary string (precision of solution)
x_range = (-10, 10)

# Step 3: Helper functions
def random_individual():
    """Create a random individual within the range."""
    return random.uniform(x_range[0], x_range[1])

def create_initial_population():
    """Create an initial population of random individuals."""
    return [random_individual() for _ in range(population_size)]

def evaluate_population(population):
    """Evaluate the fitness of the population."""
    return [fitness_function(individual) for individual in population]

def selection(population, fitness):
    """Select two individuals from the population based on fitness (roulette wheel)."""
    total_fitness = sum(fitness)
    probabilities = [f/total_fitness for f in fitness]
    selected = random.choices(population, probabilities, k=2)
    return selected

def crossover(parent1, parent2):
    """Perform crossover on two parents to create two offspring."""
    if random.random() < crossover_rate:
        alpha = random.random()  # Blend crossover
        child1 = alpha * parent1 + (1 - alpha) * parent2
        child2 = alpha * parent2 + (1 - alpha) * parent1
    else:
        child1, child2 = parent1, parent2  # No crossover, just copy parents
    return child1, child2

def mutate(individual):
    """Apply mutation to an individual with a certain mutation rate."""
    if random.random() < mutation_rate:
        # Mutate by adding a small random change
        mutation_value = random.uniform(-1, 1)
        individual += mutation_value
        # Ensure the individual stays within bounds
        individual = min(max(individual, x_range[0]), x_range[1])
    return individual

# Step 8: Genetic Algorithm Main Loop
def genetic_algorithm():
    population = create_initial_population()
    for generation in range(generations):
        # Step 4: Evaluate fitness of the population
        fitness = evaluate_population(population)

        # Track the best individual in this generation
        best_individual = population[fitness.index(max(fitness))]
        best_fitness = max(fitness)
        print(f"Generation {generation+1}: Best fitness = {best_fitness}, Best individual = {best_individual}")

        # Step 5: Create a new population via selection, crossover, and mutation
        new_population = []
        while len(new_population) < population_size:
            # Step 5: Select two parents
            parent1, parent2 = selection(population, fitness)

            # Step 6: Perform crossover
            child1, child2 = crossover(parent1, parent2)

            # Step 7: Apply mutation
            child1 = mutate(child1)
            child2 = mutate(child2)

            new_population.extend([child1, child2])

        # Limit the new population to the desired size
        population = new_population[:population_size]

    # Step 9: Output the best solution found
    fitness = evaluate_population(population)
    best_individual = population[fitness.index(max(fitness))]
    best_fitness = max(fitness)
    return best_individual, best_fitness

# Running the Genetic Algorithm
best_solution, best_fitness = genetic_algorithm()
print(f"Best solution found: x = {best_solution}, f(x) = {best_fitness}")


Generation 1: Best fitness = 62.780089417769965, Best individual = 8.251322983508583
Generation 2: Best fitness = 62.780089417769965, Best individual = 8.251322983508583
Generation 3: Best fitness = 62.780089417769965, Best individual = 8.251322983508583
Generation 4: Best fitness = 62.780089417769965, Best individual = 8.251322983508583
Generation 5: Best fitness = 62.780089417769965, Best individual = 8.251322983508583
Generation 6: Best fitness = 62.663678790903354, Best individual = 8.26144315706858
Generation 7: Best fitness = 62.62278522584213, Best individual = 8.264850245287846
Generation 8: Best fitness = 62.61746470193554, Best individual = 8.265288339002762
Generation 9: Best fitness = 62.803423311911395, Best individual = 8.249211643661619
Generation 10: Best fitness = 62.803423311911395, Best individual = 8.249211643661619
Best solution found: x = 8.249211643661619, f(x) = 62.803423311911395


In [ ]:
import random
import numpy as np

# Step 1: Define the distance matrix for 10 cities
def generate_distance_matrix(num_cities=10):
    """Generates a random distance matrix for num_cities."""
    matrix = np.random.randint(1, 100, size=(num_cities, num_cities))
    np.fill_diagonal(matrix, 0)  # Distance from a city to itself is 0
    return matrix

# Fitness function: Total distance of the tour
def fitness_function(individual, distance_matrix):
    """Calculate the total distance of the path described by the individual."""
    total_distance = sum(distance_matrix[individual[i]][individual[i + 1]] for i in range(len(individual) - 1))
    total_distance += distance_matrix[individual[-1]][individual[0]]  # Return to start
    return 1 / total_distance  # Inverse of distance, because lower distance = higher fitness

# Step 2: Initialize population (a list of random city tours)
def create_initial_population(population_size, num_cities):
    """Creates an initial population of random tours."""
    return [list(np.random.permutation(num_cities)) for _ in range(population_size)]

# Step 3: Selection using tournament selection
def tournament_selection(population, fitness_values, k=3):
    """Selects the best individual from a random sample of the population."""
    selected = random.sample(list(zip(population, fitness_values)), k)
    return max(selected, key=lambda x: x[1])[0]

# Step 4: Crossover using partially matched crossover (PMX)
def pmx_crossover(parent1, parent2):
    """Performs Partially Matched Crossover (PMX) on two parents."""
    size = len(parent1)
    child1, child2 = [-1] * size, [-1] * size
    p1, p2 = sorted(random.sample(range(size), 2))  # Random crossover points
    child1[p1:p2], child2[p1:p2] = parent1[p1:p2], parent2[p1:p2]

    def fill_child(child, parent):
        for i in range(size):
            if child[i] == -1:
                for gene in parent:
                    if gene not in child:
                        child[i] = gene
                        break
    fill_child(child1, parent2)
    fill_child(child2, parent1)
    return child1, child2

# Step 5: Mutation by swapping two cities in the tour
def mutate(individual, mutation_rate=0.01):
    """Swaps two cities in the tour with a given mutation rate."""
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]  # Swap cities
    return individual

# Step 6: Genetic Algorithm Main Loop
def genetic_algorithm_tsp(distance_matrix, population_size=50, generations=10, mutation_rate=0.01):
    num_cities = len(distance_matrix)
    population = create_initial_population(population_size, num_cities)

    for generation in range(generations):
        # Step 7: Evaluate fitness of the population
        fitness_values = [fitness_function(individual, distance_matrix) for individual in population]

        # Track the best individual in this generation
        best_individual = population[fitness_values.index(max(fitness_values))]
        best_fitness = max(fitness_values)
        print(f"Generation {generation+1}: Best fitness = {best_fitness:.4f}, Best individual = {best_individual}")

        # Step 8: Create a new population
        new_population = []
        while len(new_population) < population_size:
            # Step 9: Selection
            parent1 = tournament_selection(population, fitness_values)
            parent2 = tournament_selection(population, fitness_values)

            # Step 10: Crossover
            child1, child2 = pmx_crossover(parent1, parent2)

            # Step 11: Mutation
            new_population.append(mutate(child1, mutation_rate))
            if len(new_population) < population_size:
                new_population.append(mutate(child2, mutation_rate))

        population = new_population[:population_size]

    # Step 12: Output the best solution found
    fitness_values = [fitness_function(individual, distance_matrix) for individual in population]
    best_individual = population[fitness_values.index(max(fitness_values))]
    best_fitness = max(fitness_values)

    return best_individual, 1 / best_fitness  # Return the best tour and its total distance

# Running the Genetic Algorithm on a randomly generated TSP problem
distance_matrix = generate_distance_matrix(10)  # 10 cities

best_tour, best_distance = genetic_algorithm_tsp(distance_matrix, generations=10)  # 10 generations
print(f"Best tour found: {best_tour}")
print(f"Total distance of the best tour: {best_distance:.2f}")


Generation 1: Best fitness = 0.0038, Best individual = [2, 7, 6, 9, 1, 3, 0, 4, 5, 8]
Generation 2: Best fitness = 0.0043, Best individual = [2, 7, 6, 3, 1, 8, 9, 0, 4, 5]
Generation 3: Best fitness = 0.0040, Best individual = [2, 7, 5, 3, 1, 8, 9, 0, 4, 6]
Generation 4: Best fitness = 0.0041, Best individual = [2, 7, 6, 4, 5, 1, 3, 9, 0, 8]
Generation 5: Best fitness = 0.0041, Best individual = [2, 6, 7, 5, 8, 1, 3, 9, 0, 4]
Generation 6: Best fitness = 0.0044, Best individual = [6, 2, 9, 5, 1, 3, 8, 4, 0, 7]
Generation 7: Best fitness = 0.0048, Best individual = [2, 6, 7, 5, 1, 3, 8, 9, 0, 4]
Generation 8: Best fitness = 0.0048, Best individual = [2, 6, 7, 5, 1, 3, 8, 9, 0, 4]
Generation 9: Best fitness = 0.0048, Best individual = [6, 2, 7, 5, 1, 3, 8, 9, 0, 4]
Generation 10: Best fitness = 0.0048, Best individual = [2, 6, 7, 5, 1, 3, 8, 9, 0, 4]
Best tour found: [5, 2, 7, 6, 1, 3, 8, 9, 0, 4]
Total distance of the best tour: 205.00


In [ ]:
import random
import numpy as np

# Step 1: Define the distance matrix for 10 cities
def generate_distance_matrix(num_cities=10):
    """Generates a random distance matrix for num_cities."""
    matrix = np.random.randint(1, 100, size=(num_cities, num_cities))
    np.fill_diagonal(matrix, 0)  # Distance from a city to itself is 0
    return matrix

# Fitness function: Total distance of the tour
def fitness_function(individual, distance_matrix):
    """Calculate the total distance of the path described by the individual."""
    total_distance = sum(distance_matrix[individual[i]][individual[i + 1]] for i in range(len(individual) - 1))
    total_distance += distance_matrix[individual[-1]][individual[0]]  # Return to start
    return 1 / total_distance  # Inverse of distance, because lower distance = higher fitness

# Step 2: Initialize population (a list of random city tours)
def create_initial_population(population_size, num_cities):
    """Creates an initial population of random tours."""
    return [list(np.random.permutation(num_cities)) for _ in range(population_size)]

# Step 3: Selection using tournament selection
def tournament_selection(population, fitness_values, k=3):
    """Selects the best individual from a random sample of the population."""
    selected = random.sample(list(zip(population, fitness_values)), k)
    return max(selected, key=lambda x: x[1])[0]

# Step 4: Crossover using partially matched crossover (PMX)
def pmx_crossover(parent1, parent2):
    """Performs Partially Matched Crossover (PMX) on two parents."""
    size = len(parent1)
    child1, child2 = [-1] * size, [-1] * size
    p1, p2 = sorted(random.sample(range(size), 2))  # Random crossover points
    child1[p1:p2], child2[p1:p2] = parent1[p1:p2], parent2[p1:p2]

    def fill_child(child, parent):
        for i in range(size):
            if child[i] == -1:
                for gene in parent:
                    if gene not in child:
                        child[i] = gene
                        break
    fill_child(child1, parent2)
    fill_child(child2, parent1)
    return child1, child2

# Step 5: Mutation by swapping two cities in the tour
def mutate(individual, mutation_rate=0.01):
    """Swaps two cities in the tour with a given mutation rate."""
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]  # Swap cities
    return individual

# Step 6: Genetic Algorithm Main Loop
def genetic_algorithm_tsp(distance_matrix, population_size=50, generations=10, mutation_rate=0.01):
    num_cities = len(distance_matrix)
    population = create_initial_population(population_size, num_cities)

    for generation in range(generations):
        # Step 7: Evaluate fitness of the population
        fitness_values = [fitness_function(individual, distance_matrix) for individual in population]

        # Track the best individual in this generation
        best_individual = population[fitness_values.index(max(fitness_values))]
        best_fitness = max(fitness_values)
        print(f"Generation {generation+1}: Best fitness = {best_fitness:.4f}, Best individual = {best_individual}")

        # Step 8: Create a new population
        new_population = []
        while len(new_population) < population_size:
            # Step 9: Selection
            parent1 = tournament_selection(population, fitness_values)
            parent2 = tournament_selection(population, fitness_values)

            # Step 10: Crossover
            child1, child2 = pmx_crossover(parent1, parent2)

            # Step 11: Mutation
            new_population.append(mutate(child1, mutation_rate))
            if len(new_population) < population_size:
                new_population.append(mutate(child2, mutation_rate))

        population = new_population[:population_size]

    # Step 12: Output the best solution found
    fitness_values = [fitness_function(individual, distance_matrix) for individual in population]
    best_individual = population[fitness_values.index(max(fitness_values))]
    best_fitness = max(fitness_values)

    return best_individual, 1 / best_fitness  # Return the best tour and its total distance

# Running the Genetic Algorithm on a randomly generated TSP problem
distance_matrix = generate_distance_matrix(10)  # 10 cities

best_tour, best_distance = genetic_algorithm_tsp(distance_matrix, generations=10)  # 10 generations
print(f"Best tour found: {best_tour}")
print(f"Total distance of the best tour: {best_distance:.2f}")


Generation 1: Best fitness = 0.0036, Best individual = [9, 8, 5, 7, 4, 6, 0, 1, 2, 3]
Generation 2: Best fitness = 0.0040, Best individual = [9, 8, 5, 6, 7, 4, 0, 1, 2, 3]
Generation 3: Best fitness = 0.0036, Best individual = [9, 8, 5, 7, 4, 6, 0, 1, 2, 3]
Generation 4: Best fitness = 0.0036, Best individual = [9, 8, 5, 7, 4, 6, 0, 1, 2, 3]
Generation 5: Best fitness = 0.0042, Best individual = [8, 4, 5, 0, 1, 2, 3, 7, 6, 9]
Generation 6: Best fitness = 0.0040, Best individual = [9, 8, 5, 7, 4, 0, 1, 6, 2, 3]
Generation 7: Best fitness = 0.0049, Best individual = [9, 8, 7, 4, 5, 6, 0, 1, 2, 3]
Generation 8: Best fitness = 0.0049, Best individual = [9, 8, 7, 4, 5, 6, 0, 1, 2, 3]
Generation 9: Best fitness = 0.0049, Best individual = [9, 8, 7, 4, 5, 6, 0, 1, 2, 3]
Generation 10: Best fitness = 0.0044, Best individual = [7, 4, 5, 6, 9, 8, 0, 1, 2, 3]
Best tour found: [7, 8, 4, 5, 6, 9, 0, 1, 2, 3]
Total distance of the best tour: 189.00
